In [2]:
import requests
from io import BytesIO
from zipfile import ZipFile
from xml.etree.ElementTree import parse
import pandas as pd
import numpy as np

In [3]:


# Zip FILE (binary)

api_key = 'fc1fe4dfc0ac9df1cdaa3fa02e520e73ad64fc2e'
base_url = 'https://opendart.fss.or.kr/api/corpCode.xml' 
crtfc_key = api_key

url = base_url + '?' + 'crtfc_key' + '=' + crtfc_key   # api키를 통해서 zip파일을 다운로드 할 URL 생성

res = requests.get(url)

with ZipFile(BytesIO(res.content)) as zipfile:
    zipfile.extractall('c:\\corpCode')              # c드라이브에 Dart에서 제공한 corpCode Zip파일 다운로드

xmlTree = parse('c:\\corpCode\corpCode.xml')    #xml 파일 불러내기
    
root = xmlTree.getroot()       # XML파일은 Tree형태로 Root하기  (root print하면 -> <Element 'result' at 0x000002647C0FE778>)
list_1 = root.findall('list')  # root의 list들을 추출하기 (list_1 print하면 -> <Element 'list' at 0x0000026414C955E8> 형태로 나열 )

In [4]:
## 문제1. Elemnet형태로 짜여진 list_1의 텍스트들을 뽑아내기 위해선 아래와 같이 findtext 함수를 이용해 각각 column명을 적어줘야하는 불편함이 있음.

# print(list_1[0].findtext('corp_code'))
# print(list_1[0].findtext('corp_name'))
# print(list_1[0].findtext('stock_code'))
# print(list_1[0].findtext('modify_date'))

In [20]:
## 문제1을 해결하기 위해 for문을 활용하여 dict구조로 변환하여 DataFrame 형태로 만들어 주기.

empty_dict = {}
# empty_dict[list[1].findtext('corp_name')] = list[1].findtext('stock_code')
# empty_dict[list[2].findtext('corp_name')] = list[2].findtext('stock_code')
# list[i].findtext[('corp_name')] = list[i].findtext('stock_code')

# last_date_2021_1Q = 20210101
# last_date_2021_2Q = 20210401


for i in range(len(list_1)):  # dict 형태로 변환, stock 코드가 있으면 dict에 기업명(key)에 두 개의 valuse 종목코드와 기업코드를 넣는다.
    if (list_1[i].findtext('stock_code') != ' ' and list_1[i].findtext('modify_date') > "20200101"):\
        # ▲stock 코드가 있거나, 2021년이후로 수정된 종목
        
        empty_dict[list_1[i].findtext('corp_name')] = [str(list_1[i].findtext('stock_code')),str(list_1[i].findtext('corp_code'))
                                                      ,str(list_1[i].findtext('modify_date'))]\
        # ▲stock 코드가 있거나, 2021년이후로 수정된 종목
                                                       
# empty_dict
        
df = pd.DataFrame(empty_dict).transpose()   # 행열 전환
df.reset_index(inplace=True)                # index 하기
df.columns = ['종목명','종목코드','기업코드','수정일']   # 열 이름 부여
df

,종목명,종목코드,기업코드,수정일
0,미래에셋대우기업인수목적1호,265480,01199985,20200103
1,한컴지엠디,077280,00473400,20200116
2,한국가스공사,036460,00261285,20200109
3,한화수성기업인수목적,265920,01201581,20200115
4,비지스틸,179440,00872850,20200207
...,...,...,...,...
2621,컨버즈,109070,00753643,20210513
2622,텔라움,047730,00304401,20210506
2623,뉴파워프라즈마,144960,00521390,20210602
2624,에이디테크놀로지,200710,00657002,20210602


In [21]:
df[df['종목명'] == '카카오']

,종목명,종목코드,기업코드,수정일
52,카카오,035720,00258801,20201204


In [6]:
df.to_csv("test19.csv")

In [7]:
# df2 = pd.read_csv('test18.csv', usecols = ['종목명','종목코드','기업코드','수정일'], converters={'종목코드':str,'기업코드':str})
df2 = pd.read_csv('test18.csv')

# df2 = pd.read_excel('test17.xlsx', encoding='utf-8')
df2
# df2.loc[[0,'기업코드']]

,Unnamed: 0,종목명,종목코드,기업코드,수정일
0,0,지놈앤컴퍼니,314130,1276026,20210216
1,1,웹젠,69080,405320,20210215
2,2,우리넷,115440,664048,20210217
3,3,마이크로디지탈,305090,1267967,20210217
4,4,나노,187790,994994,20210216
...,...,...,...,...,...
1976,1976,컨버즈,109070,753643,20210513
1977,1977,텔라움,47730,304401,20210506
1978,1978,뉴파워프라즈마,144960,521390,20210602
1979,1979,에이디테크놀로지,200710,657002,20210602
